# Trade Algorithm to Automate Short Position Trades Using Daily Patterns


Only allowed short positions (selling first, buying later)


Max. stop loss 7% of initial investment


Max. take profit 14%-21% of initial investment (2:1/3:1 risk-reward ratio depending on leverage)


Support and Resistance levels calculated using anticipated pct. change in price over last 5 days


# Key Considerations Added to Simplified Structure:
## 1. Get historical data of the asset(s)
1.1. Verify data quality and check for missing values/outliers.


1.2. Account for trading volumes, volatility, and any specific asset-related adjustments.


1.3. Pull leverage-related information.


## 2. Calculate entry and exit points
2.1. Calculate anticipated pct. change in the price over the last 5 trading days.


2.2. Calculate support and resistance levels.


2.3. Ensure support/resistance levels align with leverage and volatility.
## 3. Risk Management
3.1. Set stop loss (max 7%) and take profit (between 14%-21%) based on risk-reward ratio.


3.2. Calculate position size that adheres to max stop loss and keeps total risk within acceptable boundaries.


3.3. Take leverage into consideration when calculating actual exposure and risk.
## 4. Execute the trade using Kraken REST API
4.1. Set up connection with Kraken API (handle authentication).


4.2. Monitor trade after execution for any trailing stop conditions or adjustments.


4.3. Log trades, including entries, exits, wins, and losses for further analysis. -->

In [11]:
# Import necessary libraries
import numpy as np
import pandas as pd
import requests
import datetime
import hashlib
import hmac
import base64
import time
import tensorflow as tf
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import urllib.parse

# Import config file
import config as cfg

In [12]:
api_key = cfg.api_key
private_key = cfg.api_priv

## API Fetchin

In [13]:
api_url = "https://api.kraken.com"

# Create dictionary to map API endpoints to their respective names and respective request types
api_endpoints = {
    'Assets': '/0/public/Assets',
    'AssetPairs': '/0/public/AssetPairs',
    'Ticker': '/0/public/Ticker',
    'OHLC': '/0/public/OHLC',
    'default OHLC': '/0/public/OHLC',

    'Balance': '/0/private/Balance',
    'ExtendedBalance': '/0/private/BalanceEx',
    'Ledgers': '/0/private/Ledgers',
    'QueryLedgers': '/0/private/QueryLedgers',
    'TradeVolume': '/0/private/TradeVolume',
    'TradesHistory': '/0/private/TradesHistory',
    'OpenOrders': '/0/private/OpenOrders',

}

# Function to generate a nonce
def generate_nonce():
    nonce = str(int(1000 * time.time()))
    return nonce

# Function to get Kraken signature
def get_kraken_signature(urlpath, data, secret):
    postdata = urllib.parse.urlencode(data)
    encoded = (str(data['nonce']) + postdata).encode()
    message = urlpath.encode() + hashlib.sha256(encoded).digest()

    mac = hmac.new(base64.b64decode(secret), message, hashlib.sha512)
    sigdigest = base64.b64encode(mac.digest())
    return sigdigest.decode() 

# Function to make Kraken API request
def kraken_request(uri_path, data, api_key, api_sec, headers=None):
    if headers is None:
        headers = {}

    headers['API-Key'] = api_key
    headers['API-Sign'] = get_kraken_signature(uri_path, data, api_sec)
    req = requests.post((api_url + uri_path), headers=headers, data=data)
    return req

#Function to make more non-trivial Kraken API get request
def kraken_get_request(uri_path, data=None, headers=None):
    if uri_path != api_endpoints['OHLC']:
        if headers is None:
            headers = {
                'Accept': 'application/json'
            }
        
        headers.update(headers)
        req = requests.get((api_url + uri_path), headers=headers, data=data)
    elif uri_path == api_endpoints['OHLC']:
        temp_endpoint = api_endpoints['OHLC'] + '?pair=' + data['pair'] + '&interval=' + data['interval']
        req = requests.get((api_url + temp_endpoint), headers=headers)
    return req


In [23]:
# Possible intervals: 1, 5, 15, 30, 60, 240, 1440, 10080, 21600 in minutes i.e., 1 minute, 5 minutes, 15 minutes, 30 minutes, 1 hour, 4 hours, 1 day, 1 week, 1 month
# Possible tenures: 1D (1440), 7D (10080), 1M (43200), 3M (129600), 6M (259200), 1Y (518400) - corresponding intervals are tenure/720 to maximize data points from a single request
possible_intervals =[1, 5, 15, 30, 60, 240, 1440, 10080, 21600]
possible_timeframes = {'1D': 1440, '7D': 10080, '1M': 43200, '3M': 129600, '6M': 259200, '1Y': 518400}

tenure = '1D'

assetPair = ['XXBTZUSD','BTCUSD','SOLUSD']


In [ ]:

# Step 2: Calculate Entry and Exit Points
anticipated_pct_change = calculate_anticipated_pct_change(historical_data)
support_level, resistance_level = calculate_support_resistance(historical_data, anticipated_pct_change)

# Step 3: Risk Management
entry_price = historical_data['Close'].iloc[-1]
risk_reward_ratio = 3  # Example risk-reward ratio
stop_loss, take_profit = calculate_risk_management_levels(entry_price, risk_reward_ratio)
position_size = calculate_position_size(total_capital, entry_price, stop_loss)

# Plot the historical data with support and resistance levels for the last 30 days, 
# and the anticipated price change for the next day
plt.figure(figsize=(12, 6))
plt.plot(historical_data['Close'], label='Close Price', color='blue', linewidth=1.5)
plt.axhline(y=support_level, color='green', linestyle='--', label='Support Level')
plt.axhline(y=resistance_level, color='red', linestyle='--', label='Resistance Level')
plt.title(f"Price Chart for {asset_symbol} (Last 30 Days)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

# Plot risk management levels: Stop Loss and Take Profit
plt.figure(figsize=(12, 6))
plt.plot(historical_data['Close'], label='Close Price', color='blue', linewidth=1.5)
plt.axhline(y=stop_loss, color='red', linestyle='--', label='Stop Loss')
plt.axhline(y=take_profit, color='green', linestyle='--', label='Take Profit')
plt.title(f"Risk Management Levels for {asset_symbol} (Stop Loss and Take Profit)")
plt.xlabel("Date")
plt.ylabel("Price")
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()

# Additional Graph: Volatility Trend
volatility = historical_data['Close'].pct_change().rolling(window=5).std() * 100  # 5-day rolling volatility in percentage
plt.figure(figsize=(12, 6))
plt.plot(volatility, label='5-Day Rolling Volatility', color='orange', linewidth=1.5)
plt.title(f"Volatility Trend for {asset_symbol} (Last 30 Days)")
plt.xlabel("Date")
plt.ylabel("Volatility (%)")
plt.grid(visible=True, linestyle='--', alpha=0.5)
plt.legend()
plt.tight_layout()
plt.show()


In [25]:
if __name__ == "__main__":
    main()

name 'assetPairs' is not defined
